<a href="https://colab.research.google.com/github/gee-community/geemap/blob/master/examples/notebooks/00_geemap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Using geemap with Google Colab

## Introduction

This notebook demonstrates how to use [geemap](https://geemap.org) with Google Colab. To use geemap and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account and follow the instructions [here](https://docs.google.com/document/d/1ZGSmrNm6_baqd8CHt33kIBWOlvkh-HLr46bODgJN1h0/edit?usp=sharing) to create a Cloud Project. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial).()

## Install geemap

The geemap package is pre-installed in Google Colab and is updated to the latest minor or major release every few weeks. If you would like to install the latest version, you can uncomment and run the following command:

In [ ]:
#%pip install -U geemap


## Import libraries

Import the Earth Engine Python API.

In [ ]:
import ee


## EE Authentication

Running the following cell will start the Earth Engine authentication. Follow the instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication) to authenticate Earth Engine.

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qfWGw0rmQqh1zjnYP9j_V49tGDlUnlQStBZPgdUIUc0&tc=DewKSbjGhBTM_eOWXJrZ9B4M-n51aRz3D5CR42JEiTQ&cc=C4j_fxW-mG_aAEi_tJMC8I9OlgwGlD65sd8DMfwqSsM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl47p6WE7KWqFx1rca__AL0LmYRKRUqCYzc5eQVvP8fAmJ8FD7-_wc

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
import geemap


## Basemap selector

In [ ]:
m = geemap.Map()
m.add("basemap_selector")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Define AOI through drawing tool





In [ ]:
m.draw_features

In [ ]:
m.draw_last_feature

In [ ]:
AOI = ee.FeatureCollection(m.draw_features)

## Define AOI through place boundaries from OSM (Alternative path)

In [ ]:

!pip install geopandas
!pip install shapely
!pip install osmnx

In [ ]:
import geopandas as gpd
import shapely.geometry
import osmnx as ox


In [ ]:
# Define the place name for Turin
place_name = "Ukraine, Kiev"

# Retrieve the boundary of Kiev from OpenStreetMap using osmnx
Kiev_boundaries = ox.geocode_to_gdf(place_name)

# Extracting bounding box coordinates
minx, miny, maxx, maxy = Kiev_boundaries.total_bounds

# Creating a GeoJSON-like geometry dictionary for the area of interest (AOI)
aoi_geom = {
    "coordinates": [
        [
            [minx, miny],
            [minx, maxy],
            [maxx, maxy],
            [maxx, miny],
            [minx, miny],
        ]
    ],
    "type": "Polygon",
}

# Converting the geometry dictionary to a Shapely geometry object
aoi_shape = shapely.geometry.shape(aoi_geom)

# Print the coordinates of the extracted bounding box
print(f"Bounding box coordinates: {aoi_shape.bounds}")
minx, miny, maxx, maxy = aoi_shape.bounds

In [ ]:
AOI = aoi_geom

## Get the data from MS Global Buildings

In [ ]:
objects = ee.data.listAssets('projects/sat-io/open-datasets/MSBuildings')
print('Assets in MS Global Buildings Footprint Folder', objects['assets'])

print(ee.FeatureCollection("projects/sat-io/open-datasets/MSBuildings/Ukraine").size())

feature = ee.FeatureCollection('projects/sat-io/open-datasets/MSBuildings/Ukraine').filterBounds(AOI)
m.centerObject(feature.first(),8)
m.addLayer(feature.style(**{'fillColor': '00000000', 'color': 'FF5500'})),{},'Ukraine'


Assets in MS Global Buildings Footprint Folder [{'type': 'TABLE', 'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/MSBuildings/Abyei', 'id': 'projects/sat-io/open-datasets/MSBuildings/Abyei', 'updateTime': '2022-05-27T07:26:51.088326Z'}, {'type': 'TABLE', 'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/MSBuildings/Afghanistan', 'id': 'projects/sat-io/open-datasets/MSBuildings/Afghanistan', 'updateTime': '2022-05-27T07:26:51.450428Z'}, {'type': 'TABLE', 'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/MSBuildings/Africa', 'id': 'projects/sat-io/open-datasets/MSBuildings/Africa', 'updateTime': '2022-05-27T07:26:51.836817Z'}, {'type': 'TABLE', 'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/MSBuildings/Albania', 'id': 'projects/sat-io/open-datasets/MSBuildings/Albania', 'updateTime': '2022-05-27T07:26:52.124487Z'}, {'type': 'TABLE', 'name': 'projects/earthengine-legacy/assets/projects/sat

(None, {}, 'Ukraine')

## Export the data to drive as GeoJSON

In [ ]:

# Export a subset of collection properties: three bands and the geometry
# as GeoJSON.
task = ee.batch.Export.table.toDrive(
    collection=feature,
    description="MS_building3",
    folder="Ukraine",
    fileFormat='GeoJSON',

)
task.start()
